In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from collections import deque
from datetime import datetime

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.6)
mp_drawing = mp.solutions.drawing_utils

# Parameters
VELOCITY_THRESHOLD = 0.087   # Lowered threshold to reduce false negatives
ACCELERATION_THRESHOLD = 0.015  # New threshold for acceleration check
HISTORY_LENGTH = 8       # Increased history for smoother detection
FRAME_WIDTH = 640
FRAME_HEIGHT = 480
FPS = 15

# Track motion history
motion_history = deque(maxlen=HISTORY_LENGTH)
acceleration_history = deque(maxlen=HISTORY_LENGTH)
prev_keypoints = None
prev_velocity = 0.0

def extract_keypoints(landmarks):
    keypoints = {}
    for name in ["LEFT_WRIST", "RIGHT_WRIST", "LEFT_ELBOW", "RIGHT_ELBOW", "LEFT_ANKLE", "RIGHT_ANKLE"]:
        lm = mp_pose.PoseLandmark[name].value
        keypoints[name] = np.array([landmarks[lm].x, landmarks[lm].y])
    return keypoints

def calculate_average_velocity(curr_kp, prev_kp):
    velocities = []
    for k in curr_kp:
        if k in prev_kp:
            v = np.linalg.norm(curr_kp[k] - prev_kp[k])
            velocities.append(v)
    return np.mean(velocities) if velocities else 0.0

def detect_fight(frame):
    global prev_keypoints, prev_velocity
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if not results or not results.pose_landmarks:
        return "Unknown", 0.0, 0.0, None

    landmarks = results.pose_landmarks.landmark
    curr_keypoints = extract_keypoints(landmarks)

    if prev_keypoints is None:
        prev_keypoints = curr_keypoints
        return "Safe", 0.0, 0.0, results.pose_landmarks

    velocity = calculate_average_velocity(curr_keypoints, prev_keypoints)
    acceleration = abs(velocity - prev_velocity)

    motion_history.append(velocity)
    acceleration_history.append(acceleration)

    avg_velocity = np.mean(motion_history)
    avg_acceleration = np.mean(acceleration_history)

    prev_keypoints = curr_keypoints
    prev_velocity = velocity

    if avg_velocity > VELOCITY_THRESHOLD and avg_acceleration > ACCELERATION_THRESHOLD:
        return "🚨 FIGHT DETECTED", avg_velocity, avg_acceleration, results.pose_landmarks
    return "✅ Safe", avg_velocity, avg_acceleration, results.pose_landmarks

def main():
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)
    cap.set(cv2.CAP_PROP_FPS, FPS)

    if not cap.isOpened():
        print("❌ Camera not accessible")
        return

    print("✅ Camera started. Press 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        status, velocity, acceleration, pose_landmarks = detect_fight(frame)
        color = (0, 0, 255) if "FIGHT" in status else (0, 255, 0)

        # Draw status bar
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cv2.rectangle(frame, (0, 0), (FRAME_WIDTH, 80), color, -1)
        cv2.putText(frame, f"{status} | Vel: {velocity:.4f} | Acc: {acceleration:.4f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 255, 255), 2)
        cv2.putText(frame, f"Time: {timestamp}", (10, 65),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 255, 255), 1)

        if pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("📷 Fight Detection - MediaPipe Only", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

